# <font color=red>Innsending av mva-melding til altinn3</font>

Test-script for å laste opp en mva-melding til altinn3-appen på tt02 miljøet i Altinn Studio.

## <font color=red>Sjekke at du har installert de nødvendige bibliotekene.</font>

In [ ]:
try:
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from Steg import load_xml_files, get_altinntoken, get_instance, upload_vatreturnfiling, upload_vatreturn
    from Steg import upload_attachment, log_in_idporten, update_process, create_instance, delete_attachment
    from Steg.FilingServices import Environment, LoginMethod, printing, App
    from Steg.validation import validate_vat_return, validate_example_file
except ImportError as e:
    try:
        print("Mangler en avhengighet, installer dem via pip")
        !pip install python-jose[cryptography] cryptography
        !pip install xmltodict
        !pip install requests
        import xmltodict
        import requests
        from Steg import load_xml_files, get_altinntoken, get_instance, upload_vatreturnfiling, upload_vatreturn
        from Steg import upload_attachment, log_in_idporten, update_process, create_instance, delete_attachment
        from Steg.FilingServices import Environment, LoginMethod, printing, App
        from Steg.validation import validate_vat_return, validate_example_file
    except ImportError as err:
        print("Mangler en avhengighet, installer dem via pip")
        !python3 -m pip install python-jose[cryptography] cryptography
        !python3 -m pip install 'python-jose[cryptography]'
        !python3 -m pip install xmltodict
        !python3 -m pip install requests
        import xmltodict
        import requests
        from Steg import load_xml_files, get_altinntoken, get_instance, upload_vatreturnfiling, upload_vatreturn
        from Steg import upload_attachment, log_in_idporten, update_process, create_instance, delete_attachment
        from Steg.FilingServices import Environment, LoginMethod, printing, App
        from Steg.validation import validate_vat_return, validate_example_file

## <font color=red>Definere variabler</font>

In [ ]:
slette_forrige_vedlegg = False  # Hvis man vil slette et vedlegg man har sendt inn ved feil.
validere = True  # Hvis man vil kjøre gjennom valideringen.

# Trenger ikke å gjøre noe med.
miljo = Environment.tt02  # tt02
innloggings_metode = LoginMethod.idporten  # idporten
domene = miljo.value  # Domenet til appen på Altinn Studio
app = App.ETM2.value  # Hvilken app repository som kjøres

# Fil navn på MvaMeldingInnsending ("konvolutt") mvamelding som ønskes å lastes opp.
# Må ligge i ./eksempler/konvolutt/ og ./eksempler/melding/
MvaMeldingInnsending_filnavn = "mvakonvolutt1.xml"
mvamelding_filnavn = "mvakode3.xml"

# Legg inn org_nummer du ønsker å sende inn for.
# Merk at personen som skal sende inn må ha nødvendig roller/rettigheter i altinn studio appen, for å få lov til å sende inn.
# org_nummer må være det samme som org_nummeret i MvaMeldingInnsending ("konvolutt") og mvamelding .xml filene
org_nummer = "123456789"

## <font color=red>Definere MvaMelding og MvaMeldingInnsending.</font>

In [ ]:
melding_xml, konvolutt_xml = load_xml_files.get(MvaMeldingInnsending_filnavn, mvamelding_filnavn)

## <font color=red>Generer ID-porten token</font>
Tokenet er gyldig i 300 sekunder, rekjør denne biten om du ikke har kommet frem til altinntoken innen den tid.

In [ ]:
header = log_in_idporten.get_idtoken()

## <font color=red>Valider meldingen(Valgfritt)</font>

In [ ]:
if validere:
  validering = validate_vat_return(dict(header), xml=melding_xml)

## <font color=red>Hente AltinnToken</font>

In [ ]:
altinn_token = get_altinntoken.get(header)

## <font color=red>Opprett ny instans</font>

In [ ]:
instans = create_instance.create(domene, altinn_token, app, org_nummer)

partyId = instans['instanceOwner']['partyId']
instanceGuid = instans['data'][0]['instanceGuid']
dataId = instans['data'][0]['id']

instansUrl = domene + instans['appId'] + "/instances/" + partyId + "/" + instanceGuid
mvaMeldingInnsendingUrl  = instansUrl + "/data/" + dataId
mvaMeldingUrl = instansUrl + "/data?dataType=mvamelding"
mvaMeldingVedleggUrl  = instansUrl + "/data?dataType=vedlegg"

## <font color=red>Last opp MvaMeldingInnsending</font>

In [ ]:
upload_vatreturnfiling.upload(mvaMeldingInnsendingUrl, konvolutt_xml, altinn_token)

## <font color=red>Last opp MvaMelding</font>

In [ ]:
upload_vatreturn.upload(mvaMeldingUrl, melding_xml, altinn_token)

## <font color=red>Last opp Vedlegg(valgfritt)</font>

#### .xml vedlegg

In [ ]:
xml_vedlegg = upload_attachment.upload(mvaMeldingVedleggUrl, "mva-vedlegg.xml", "text/xml" , altinn_token)

#### .pdf vedlegg

In [ ]:
pdf_vedlegg = upload_attachment.upload(mvaMeldingVedleggUrl, "pdf-vedlegg.pdf", "application/pdf", altinn_token)

#### .png vedlegg

In [ ]:
png_vedlegg = upload_attachment.upload(mvaMeldingVedleggUrl, "png-vedlegg.png", "image/png", altinn_token)

## <font color=red>Send Inn MvaMeldingInnsending</font>

In [ ]:
avslutteProsess = update_process.next_step(instansUrl, altinn_token)

## <font color=red>Hente Instansen</font>

In [ ]:
instans = get_instance.get(instansUrl, altinn_token)